In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

In [32]:
# Load data
# Replace 'file.xlsx' with the path to your file
data = pd.read_excel('Online Retail.xlsx',parse_dates=['InvoiceDate'])




In [34]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [36]:
# Compute total sales per product
data['TotalSales'] = data['Quantity'] * data['UnitPrice']
data = data.groupby(['InvoiceDate', 'StockCode']).agg({'TotalSales': 'sum'}).reset_index()

In [38]:
# Feature engineering
data['DayOfWeek'] = data['InvoiceDate'].dt.dayofweek
data['Month'] = data['InvoiceDate'].dt.month

In [40]:
# Create lag features for demand prediction
data['Lag1'] = data.groupby('StockCode')['TotalSales'].shift(1)
data['Lag7'] = data.groupby('StockCode')['TotalSales'].shift(7)
data = data.dropna()

In [42]:
# Define features and target
X = data[['Lag1', 'Lag7', 'DayOfWeek', 'Month']]
y = data['TotalSales']

In [44]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [46]:
# Train Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

In [47]:
# Predict on test set
y_pred = model.predict(X_test)

In [48]:
# Evaluate model performance
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'RMSE: {rmse}')

RMSE: 95.9608907012553


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [49]:
# Save the model
joblib.dump(model, 'demand_prediction_model.pkl')
print('Model saved to demand_prediction_model.pkl')

Model saved to demand_prediction_model.pkl
